# <font color='#5BC60C'>DASHBOARD</font> 

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="on/off the code."></form>''')



In [10]:
import plotly.graph_objects as go
from ipywidgets import widgets
import pandas as pd
%matplotlib inline
import numpy as np
import seaborn as sns
import plotly.tools as tls
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn import decomposition
%config InlineBackend.figure_format='retina'
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#Tejaswi Gorde- 111650817
#Please watch the video first.


f = pd.read_csv("airbnb_&_crime.csv")
f.head()

,Location,Crime_count,price,minimum_nights,number_of_reviews
0,Bronx,7392,66,5,124
1,Brooklyn,9734,135,6,107
2,Manhattan,8608,289,3,13
3,Queens,6821,147,12,46
4,Staten Island,1829,80,3,112


In [11]:

#initial data
f = pd.read_csv("airbnb_&_crime.csv")
print("Visualization dashboard below is created from Airbnb Vs Crime data in 5 bouroughs of New York. Let learn more about the commonly understood facts about New York data and some those we learned from these visualization.")


# <font color='#5BC60C'>DASHBOARD</font> 
#run this code to see the correlation data used for the corelaion matrix
da=f[['Crime_count','price','minimum_nights','number_of_reviews']].corr()
da    


# from matplotlib import pyplot
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


fig = make_subplots(rows=2, cols=2)

# bar graph 1
fig.add_trace(go.Bar(name='price', x=f['Location'], y=f['price']),row=1,col=1)
# fig.show()

# bar graph 2
fig.add_trace(go.Bar(name="min_night", x=f['Location'], y=f['minimum_nights']),row=1,col=2)
# fig.show()

#scatter plot
fig.add_trace(go.Scatter(name="Price/Crime",x=f['price'],y=f['Crime_count'],text=["Bronx","Brooklyn","Manhattan","Queens","Staten Island"]),row=2, col=1)


#corr matrix in heatmap
fig.add_trace(go.Heatmap(z=[[1.000000,0.472697,0.202336,-0.299542],[0.472697,1.000000,-0.112712,-0.909237],
                                   [0.202336,-0.112712,1.000000,-0.199395],[-0.299542,-0.909237,-0.199395,1.000000]]
                               ,x=['crime','price','min_night','reviews'],
                                y=['crime','price','min_night','reviews'],showscale = False),row=2,col=2)

fig.update_layout(height=1000, width=1000, title_text="Visualizations")

fig.show()

#parallel axis
fig = go.Figure(data=
    go.Parcoords(
        line = dict(color ='darkblue',
                   colorscale = [[0,'purple'],[0.5,'lightseagreen'],[1,'gold']]),
        dimensions = list([
#             dict(range = [0,8],
#                 constraintrange = [4,8],
#                 label = 'Location', values = f['Location']),
            dict(range = [0,10000],
                label = 'Crime count', values = f['Crime_count']),
            dict(range = [0,300],
                label = 'Price', values = f['price']),
            dict(range = [0,15],
                label = 'Minimum nights', values = f['minimum_nights']),
            dict(range = [0,150],
                label = 'Reviews', values = f['number_of_reviews'])
        ])
    )
)

fig.update_layout(
    plot_bgcolor = 'white',
    paper_bgcolor = 'white'
)

fig.show()




#Brushing Linking
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import ipywidgets as widgets
init_notebook_mode(connected=True)

cars_df = pd.read_csv("airbnb_&_crime.csv")

# Build parcats dimensions
categorical_dimensions = [
  'price',
  'minimum_nights',
  'Crime_count','number_of_reviews'
];

dimensions = [
    dict(values=cars_df[label], label=label)
    for label in categorical_dimensions
]

# Build colorscale
color = np.zeros(len(cars_df), dtype='uint8')
colorscale = [[0, 'gray'], [0.33, 'gray'],
              [0.33, 'firebrick'], [0.66, 'firebrick'],
              [0.66, 'blue'], [1.0, 'blue']];
cmin = -0.5
cmax = 2.5

# Build figure as FigureWidget
fig = go.FigureWidget(
    data=[
        go.Scatter(
            x=cars_df.Location,
            y=cars_df['Crime_count'],
            marker={'color': color,
                    'cmin': cmin,
                    'cmax': cmax,
                    'colorscale': colorscale,
                    'showscale': True,
                    'colorbar': {'tickvals': [0, 1, 2],
                                 'ticktext': ['None', 'Red', 'Blue']}
                   },
            mode='markers'),

        go.Parcats(
            domain={'y': [0, 0.4]},
            dimensions=dimensions,
            line={
                'colorscale': colorscale,
                'cmin': cmin,
                'cmax': cmax,
                'color': color,
                'shape': 'hspline'})
    ],
    layout=go.Layout(
        height=800,
        xaxis={'title': 'Location'},
        yaxis={'title': 'Crime count',
               'domain': [0.6, 1]},
        dragmode='lasso',
        hovermode='closest')
)

# Build color selection widget
color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Blue'],
    index=1,
    description='Brush Color:',
    disabled=False,
)

# Update color callback
def update_color(trace, points, state):
    # Compute new color array
    new_color = np.array(fig.data[0].marker.color)
    new_color[points.point_inds] = color_toggle.index

    with fig.batch_update():
        # Update scatter color
        fig.data[0].marker.color = new_color

        # Update parcats colors
        fig.data[1].line.color = new_color

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
# and parcats click
fig.data[1].on_click(update_color)

# Display figure
widgets.VBox([color_toggle, fig])



Visualization dashboard below is created from Airbnb Vs Crime data in 5 bouroughs of New York. Let learn more about the commonly understood facts about New York data and some those we learned from these visualization.
